# 📋 GUIA COMPLETO CODEC - MIGRAÇÃO PARA CUSTOMTKINTER

**Objetivo:** Remover 100% de ttkbootstrap dos módulos legados e migrar para CustomTkinter.

**Status:** Widget DatePicker CTk criado ✅

---

## 🎯 MÓDULOS A MIGRAR

| Módulo | Arquivos | Status |
|--------|----------|--------|
| **tasks** | 1 | ✅ MIGRADO |
| **passwords** | 3 | ⏳ PENDENTE |
| **lixeira** | 1 | ⏳ PENDENTE |
| **cashflow** | 1 | ⏳ PENDENTE |
| **anvisa** | 3 | ⏳ PENDENTE |

## 📐 PADRÃO DE SUBSTITUIÇÃO

### **1. Imports**

```python
# ❌ ANTES
import ttkbootstrap as tb
from ttkbootstrap.dialogs import Messagebox
from ttkbootstrap.constants import BOTH, LEFT
from ttkbootstrap.widgets import DateEntry

# ✅ DEPOIS
from src.ui.ctk_config import ctk  # SSoT
from tkinter import messagebox
from src.ui.widgets import CTkDatePicker
```

### **2. Widgets Principais**

| ttkbootstrap | CustomTkinter | Notas |
|--------------|---------------|-------|
| `tb.Toplevel` | `ctk.CTkToplevel` | Janelas secundárias |
| `tb.Frame` | `ctk.CTkFrame` | Container (`fg_color="transparent"`) |
| `tb.Label` | `ctk.CTkLabel` | Texto estático |
| `tb.Button` | `ctk.CTkButton` | Botões |
| `tb.Entry` | `ctk.CTkEntry` | Input texto |
| `tb.Text` | `ctk.CTkTextbox` | Texto multilinha |
| `tb.Combobox` | `ctk.CTkComboBox` | **Usar `variable=` não `textvariable=`** |
| `DateEntry` | `CTkDatePicker` | Widget customizado ✅ |
| `ttk.Treeview` | **MANTER** | Sem equivalente CTk |

### **3. CTkComboBox - IMPORTANTE**

```python
# ❌ ANTES (ttkbootstrap)
combo = tb.Combobox(
    parent,
    textvariable=var,  # ❌
    values=["A", "B"],
    state="readonly"
)
combo.current(0)  # Seleciona índice

# ✅ DEPOIS (CustomTkinter)
combo = ctk.CTkComboBox(
    parent,
    variable=var,  # ✅ variable (não textvariable)
    values=["A", "B"],
    state="readonly"
)
combo.set("A")  # Define valor diretamente
```

### **4. DateEntry → CTkDatePicker**

```python
# ❌ ANTES (ttkbootstrap)
from ttkbootstrap.widgets import DateEntry

date_widget = DateEntry(
    parent,
    dateformat="%Y-%m-%d",
    startdate=date.today()
)
date_widget.pack()
value = date_widget.entry.get()  # Acesso indireto

# ✅ DEPOIS (CustomTkinter)
from src.ui.widgets import CTkDatePicker

date_widget = CTkDatePicker(
    parent,
    dateformat="%Y-%m-%d",
    startdate=date.today()
)
date_widget.pack()
value = date_widget.get()  # Acesso direto
date_obj = date_widget.get_date()  # Retorna date object
```

### **5. Messagebox**

```python
# ❌ ANTES
from ttkbootstrap.dialogs import Messagebox
Messagebox.show_error("Mensagem", "Título", parent=self)
result = Messagebox.yesno("Pergunta", "Título", parent=self)

# ✅ DEPOIS
from tkinter import messagebox
messagebox.showerror("Título", "Mensagem", parent=self)
result = messagebox.askyesno("Título", "Pergunta", parent=self)
```

### **6. Cores e Estilos**

```python
# ❌ ANTES (bootstyle)
tb.Button(text="OK", bootstyle="success")
tb.Button(text="Cancel", bootstyle="secondary")
tb.Label(text="Info", foreground="gray")

# ✅ DEPOIS (tuple colors para light/dark)
ctk.CTkButton(
    text="OK",
    fg_color="green",
    hover_color="darkgreen"
)
ctk.CTkButton(
    text="Cancel",
    fg_color="gray",
    hover_color="darkgray"
)
ctk.CTkLabel(
    text="Info",
    text_color=("gray20", "gray80")  # (light_mode, dark_mode)
)
```

### **7. Treeview (MANTER COM MASTER)**

```python
# ✅ CORRETO (ttk.Treeview com master explícito)
from tkinter import ttk

# Embutir em CTkFrame
tree_container = ctk.CTkFrame(self)
tree_container.pack(fill="both", expand=True)

# Treeview com master explícito
tree = ttk.Treeview(
    tree_container,  # ✅ master explícito
    show="headings",
    columns=("col1", "col2")
)
tree.pack(fill="both", expand=True)

# Aplicar estilo com master
style = ttk.Style(tree_container)  # ✅ master explícito
# Ou usar ttk_compat se disponível
```

## 🔄 PROCEDIMENTO POR MÓDULO

Para cada arquivo:

### **Passo 1: Substituir Imports**
- Remover `import ttkbootstrap as tb`
- Adicionar `from src.ui.ctk_config import ctk`
- Trocar `Messagebox` por `messagebox`
- Trocar `DateEntry` por `CTkDatePicker`

### **Passo 2: Substituir Widgets**
- `tb.Toplevel` → `ctk.CTkToplevel`
- `tb.Frame` → `ctk.CTkFrame`
- `tb.Label` → `ctk.CTkLabel`
- `tb.Button` → `ctk.CTkButton`
- `tb.Entry` → `ctk.CTkEntry`
- `tb.Text` → `ctk.CTkTextbox`
- `tb.Combobox` → `ctk.CTkComboBox` (**usar `variable=`**)

### **Passo 3: Ajustar Cores**
- Remover `bootstyle=`
- Adicionar `fg_color=` e `hover_color=`
- Trocar `foreground=` por `text_color=`

### **Passo 4: CTkComboBox**
- `textvariable=` → `variable=`
- `.current(0)` → `.set(valor)`

### **Passo 5: DateEntry**
- Importar `from src.ui.widgets import CTkDatePicker`
- `DateEntry(...)` → `CTkDatePicker(...)`
- `.entry.get()` → `.get()`

### **Passo 6: Validar**
```bash
python -m compileall -q src/modules/<MODULO>
rg -n "import ttkbootstrap" src/modules/<MODULO> --type py
```

## 📝 CHECKLIST FINAL

Após migrar todos os módulos:

- [ ] ✅ Compilação geral: `python -m compileall -q src tests`
- [ ] ✅ Zero ttkbootstrap: `rg -n "import ttkbootstrap" src/modules --type py`
- [ ] ✅ SSoT mantido: `rg -n "set_appearance_mode" src --type py` (só theme_manager.py)
- [ ] ✅ Sem Style() vazio: `rg -n "^[^#\n]*\\bttk\\.Style\\(\\s*\\)" src --type py`
- [ ] ✅ Política CTk: `python scripts/validate_ctk_policy.py`
- [ ] ✅ ThemeManager: `python -c "from src.ui.theme_manager import ThemeMode; print(ThemeMode.__args__)"`

## 🎯 ARQUIVOS PARA MIGRAR

### **passwords (3 arquivos)**
1. `src/modules/passwords/views/passwords_screen.py`
2. `src/modules/passwords/views/password_dialog.py`
3. `src/modules/passwords/views/client_passwords_dialog.py`

### **lixeira (1 arquivo)**
1. `src/modules/lixeira/views/lixeira.py`

### **cashflow (1 arquivo)**
1. `src/modules/cashflow/views/fluxo_caixa_frame.py`

### **anvisa (3 arquivos)**
1. `src/modules/anvisa/views/anvisa_screen.py`
2. `src/modules/anvisa/views/anvisa_footer.py`
3. `src/modules/anvisa/views/_anvisa_history_popup_mixin.py`